## daten zusammenfügen und anpassen

In [1]:
import pandas as pd
import requests
from datetime import datetime
import os

In [2]:
#Verzeichnis anpassen

os.chdir('C:/Users/Katharina/Desktop/Weiterbildung/Bootcamp/Bootcamp/Final_project/data')
print(os.getcwd())

C:\Users\Katharina\Desktop\Weiterbildung\Bootcamp\Bootcamp\Final_project\data


In [11]:
#1. Komplette Daten einlesen mit ID
Database = pd.read_csv('Data_final_mitTimezone.csv', sep=';')

C:\Users\Katharina\AppData\Local\Temp\ipykernel_10624\1975893712.py:2: DtypeWarning: Columns (1,10,13,14,21,49,51,59,68,101) have mixed types. Specify dtype option on import or set low_memory=False.
  Database = pd.read_csv('Data_final_mitTimezone.csv', sep=';')


In [ ]:
#Zeit anpassen

In [18]:
Database['Zeitpunkt'] = Database['@LocalDate'] + ' ' + Database['@LocalTime']

In [19]:
Data_Prep =Database[['ID_DefaultCity', '@LocalDate', '@LocalTime', 'longitude', 'latitude', 'timezone', 'Zeitpunkt']].drop_duplicates()

## Anwendung Wetter API

In [23]:
#anpassunge an wetter-api

import pandas as pd
import numpy as np
import time
import requests
from datetime import datetime, timedelta
import pytz
import urllib.parse

def get_weather_data_for_df(df, wait_per_call=0.72):
    """
    Verarbeitet einen DataFrame, der mindestens folgende Spalten enthält:
      - '@LocalDate' (Format: "YYYY-MM-DD")
      - '@LocalTime' (Format: "HH:MM:SS")
      - 'Latitude'
      - 'Longitude'
      - 'Timezone' (z. B. "Europe/Berlin")
    
    Für jede Zeile wird ein API-Aufruf getätigt, wobei zwischen den Aufrufen
    ein Delay von wait_per_call Sekunden eingefügt wird. Damit werden maximal
    ca. 5000 Aufrufe pro Stunde sichergestellt.
    """
    results = []
    
    for idx, row in df.iterrows():
        date = row['@LocalDate']
        start_time = row['@LocalTime']
        latitude = row['latitude']
        longitude = row['longitude']
        timezone_str = row['timezone']  # Dynamische Zeitzone aus dem DataFrame
        
        try:
            # Lokaler Zeitpunkt: Datum + Uhrzeit in der angegebenen Zeitzone
            local_tz = pytz.timezone(timezone_str)
            local_dt = datetime.strptime(f"{date} {start_time}", "%Y-%m-%d %H:%M:%S")
            local_dt = local_tz.localize(local_dt)
            
            # Eine Stunde als Zeitraum definieren
            end_local_dt = local_dt + timedelta(hours=1)
            
            # Formatierung als ISO-String (ohne Offset, da die API über den Parameter 'timezone'
            # die Zeitzone vorgibt)
            start_datetime_str = local_dt.strftime("%Y-%m-%dT%H:%M:%S")
            end_datetime_str = end_local_dt.strftime("%Y-%m-%dT%H:%M:%S")
            local_date_str = local_dt.strftime("%Y-%m-%d")
            
            # URL-kodierte Zeitzone (z. B. "Europe/Berlin" → "Europe%2FBerlin")
            encoded_timezone = urllib.parse.quote(timezone_str)
            
            # Aufbau der API-URL; hier wird die Zeitzone dynamisch übergeben
            url = (
                f"https://archive-api.open-meteo.com/v1/archive"
                f"?latitude={latitude}&longitude={longitude}"
                f"&start_date={local_date_str}&end_date={local_date_str}"
                f"&hourly=temperature_2m,precipitation,wind_speed_10m,rain,wind_gusts_10m"
                f"&timezone={encoded_timezone}"
            )
            
            response = requests.get(url)
            response.raise_for_status()
            dataW = response.json()
            
            df_weather = pd.DataFrame(dataW['hourly'])
            # Filter: nur die Zeilen, die innerhalb des einstündigen Zeitraums liegen
            df_filtered = df_weather[
                (df_weather['time'] >= start_datetime_str) & 
                (df_weather['time'] < end_datetime_str)
            ].copy()
            
            # Zur späteren Zuordnung merken wir uns den Zeilenindex
            if not df_filtered.empty:
                df_filtered['source_row'] = idx
            
            results.append(df_filtered)
            
        except Exception as e:
            print(f"Fehler bei Zeile {idx}: {e}")
            continue
        
        # Throttling: Warten, um sowohl 600 Aufrufe pro Minute als auch maximal 5000 Aufrufe pro Stunde einzuhalten.
        # 3600 Sek / 5000 Aufrufe ≈ 0.72 sec pro Aufruf.
        time.sleep(wait_per_call)
    
    if results:
        weather_all = pd.concat(results, ignore_index=True)
        return weather_all
    else:
        return pd.DataFrame()





In [24]:
# Angenommen, 'df' ist dein DataFrame mit 10.276 Zeilen
# Aufteilen in 3 ungefähr gleich große Teile
dfs = np.array_split(Data_Prep, 3)

all_weather_results = []
for i, sub_df in enumerate(dfs, start=1):
    print(f"Verarbeite Batch {i} von {len(dfs)} (Zeilen: {len(sub_df)})...")
    weather_batch = get_weather_data_for_df(sub_df, wait_per_call=0.72)
    all_weather_results.append(weather_batch)
    
    # Optionale zusätzliche Pause zwischen den Batches, falls notwendig:
    time.sleep(60)

# Zusammenführen aller Ergebnisse
final_weather_df = pd.concat(all_weather_results, ignore_index=True)
print("Verarbeitung abgeschlossen. Gesamtanzahl abgerufener Wetter-Datensätze:", len(final_weather_df))


c:\Users\Katharina\Desktop\Weiterbildung\Bootcamp\Bootcamp\Final_project\.venv\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Verarbeite Batch 1 von 3 (Zeilen: 3426)...
Fehler bei Zeile 11108: time data '2024-06-08 nan' does not match format '%Y-%m-%d %H:%M:%S'
Fehler bei Zeile 11140: time data '2024-06-09 nan' does not match format '%Y-%m-%d %H:%M:%S'
Fehler bei Zeile 30764: time data '2023-05-12 nan' does not match format '%Y-%m-%d %H:%M:%S'
Verarbeite Batch 2 von 3 (Zeilen: 3425)...
Fehler bei Zeile 34224: 'float' object has no attribute 'upper'
Fehler bei Zeile 34228: 'float' object has no attribute 'upper'
Fehler bei Zeile 34240: 'float' object has no attribute 'upper'
Fehler bei Zeile 34252: 'float' object has no attribute 'upper'
Fehler bei Zeile 34256: 'float' object has no attribute 'upper'
Fehler bei Zeile 34268: 'float' object has no attribute 'upper'
Fehler bei Zeile 34280: 'float' object has no attribute 'upper'
Fehler bei Zeile 34288: 'float' object has no attribute 'upper'
Fehler bei Zeile 34296: 'float' object has no attribute 'upper'
Fehler bei Zeile 34304: 'float' object has no attribute 'up

In [25]:
final_weather_df.to_csv('weather_gescrapt.csv', index=False, sep=';')

In [30]:
import time
import numpy as np
import pandas as pd

# Angenommen, Data_Prep ist dein DataFrame mit 10.276 Zeilen
dfs2 = np.array_split(Data_Prep, 3)

all_weather_results2 = []
# for i, sub_df in enumerate(dfs2, start=1):
#     print(f"Verarbeite Batch {i} von {len(dfs)} (Zeilen: {len(sub_df)})...")
#     weather_batch = get_weather_data_for_df(sub_df, wait_per_call=0.72)
#     all_weather_results.append(weather_batch)
    
#     # Zusätzliche Pause zwischen den Batches, um die API-Limits einzuhalten
#     time.sleep(60)

# Nun den dritten Batch nochmal scrapen
print("Re-Scraping des dritten Batches...")
third_batch_rerun = get_weather_data_for_df(dfs[2], wait_per_call=0.72)

# Den dritten Batch in den Ergebnissen aktualisieren
all_weather_results2 = third_batch_rerun

# Zusammenführen aller Ergebnisse
#final_weather_df2 = pd.concat(all_weather_results, ignore_index=True)
#print("Verarbeitung abgeschlossen. Gesamtanzahl abgerufener Wetter-Datensätze:", len(final_weather_df))


c:\Users\Katharina\Desktop\Weiterbildung\Bootcamp\Bootcamp\Final_project\.venv\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Re-Scraping des dritten Batches...
Fehler bei Zeile 67936: time data '2018-07-06 nan' does not match format '%Y-%m-%d %H:%M:%S'
Fehler bei Zeile 71400: time data '2019-07-13 nan' does not match format '%Y-%m-%d %H:%M:%S'
Fehler bei Zeile 78960: time data '2019-08-22 nan' does not match format '%Y-%m-%d %H:%M:%S'
Fehler bei Zeile 86030: 'float' object has no attribute 'upper'
Fehler bei Zeile 86046: 'float' object has no attribute 'upper'
Fehler bei Zeile 86062: 'float' object has no attribute 'upper'
Fehler bei Zeile 86078: 'float' object has no attribute 'upper'
Fehler bei Zeile 86094: 'float' object has no attribute 'upper'
Fehler bei Zeile 86098: 'float' object has no attribute 'upper'
Fehler bei Zeile 86126: 'float' object has no attribute 'upper'
Fehler bei Zeile 86130: 'float' object has no attribute 'upper'
Fehler bei Zeile 86158: 'float' object has no attribute 'upper'
Fehler bei Zeile 86174: 'float' object has no attribute 'upper'
Fehler bei Zeile 86190: 'float' object has no 

In [31]:
all_weather_results2.to_csv('weather_result2.csv', index=False, sep=';')

### Daten zusammenfassen

In [33]:
wetter_final = pd.concat([final_weather_df, all_weather_results2])
wetter_df = wetter_final.drop_duplicates(subset=['source_row'])

- es fehlen die zeilen bei denen die Koordinaten NA waren
- daten zusammenführen mit Data_Prep bzw. Database

In [38]:
Data_Prep['source_row'] = Data_Prep.index

In [41]:
DatZ1 = Data_Prep.merge(wetter_df, left_on='source_row', right_on='source_row', how='left')

In [ ]:
#zeilen mit fehlender Uhrzeit entfernen
DatZ2 = DatZ1.dropna(subset=('@LocalTime'))



In [55]:
DatFZ = Database.merge(DatZ1[['ID_DefaultCity',  'Zeitpunkt', 'index_col', 'source_row', 'time',
       'temperature_2m', 'precipitation', 'wind_speed_10m', 'rain',
       'wind_gusts_10m']], left_on=['ID_DefaultCity', 'Zeitpunkt'],
                       right_on=['ID_DefaultCity', 'Zeitpunkt'],
                       how='left')

In [58]:
diff=DatFZ[DatFZ.duplicated(subset='ID_row', keep=False)]
#die zeilen haben keine wetter infos
DatFZ2 = DatFZ[~DatFZ.duplicated(subset='ID_row', keep=False)]

#rausnehmen


In [59]:
DatFZ2.to_csv('Daten_zusammen_final.csv', index=False, sep=';')

In [ ]:
#Variablen, die keine Rolle im Modell spielen, entfernen

In [62]:
DatFZ3 = DatFZ2.drop(['Actions', 'BuyTicketsUrl','Deadline','DefaultLocalTimeOffset','DefaultMatchFormat','DefaultTimeZone',
'DispatchMethod', 'DispatchStatus', 'EarningsCurrencyCode', 'EndDateFederationQuota',
'EntryPointsBaseDate','EntryPointsDayOffset', 'EntryPointsTemplateNo','EventAuxiliaryPersons',
'EventLogos','IsFreeEntrance', 'IsVisManaged','Logos','MatchPointsMethod',
'MaxRegisterFederation', 'MaxRegisterHost', 'MaxReserveTeams', 'MaxTeamsDispatchFederation',
'MaxTeamsDispatchHost','MaxTeamsFederation','MaxTeamsHost','MaxTeamsMainDrawFederation', 'MaxTeamsMainDrawHost',
'MinConfederationTeams', 'MinTeamsDispatchHost', 'NbTeamsFromQualification',
'NbTeamsMainDraw', 'NbTeamsQualification','NbUploads', 'NbWildCards', 'NoTemplateEarnedPoints',
'NoTemplatePrizeMoney','OrganizerCode', 'OrganizerType','Parameters', 'PreliminaryInquiryMainDraw',
'PreliminaryInquiryQualification','SeedPointsBaseDate', 'SeedPointsDayOffset', 'SeedPointsTemplateNo',
'StartDateFederationQuota', 'TechnicalEntryPointsTemplateNo','TechnicalSeedPointsTemplateNo',
'Draws', 'check', 'check2', 'NoSet', 'point', 'latitude', 'longitude', 'timezone', 'UTC_offset_hours', 
'Zeitpunkt', 'index_col','source_row','time'],axis=1)